<a href="https://colab.research.google.com/github/JoanChege/CHATBOT1/blob/main/ModelFinal5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import
!pip install pyspellchecker
!pip install contractions
!pip install symspellpy
!pip install nltk contractions pyenchant
!pip install gensim

import json
import pickle
import numpy as np
import pandas as pd
import contractions
from contractions import contractions_dict
import re
from spellchecker import SpellChecker
from symspellpy import SymSpell, Verbosity
from wordcloud import STOPWORDS

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from gensim.models import Word2Vec


import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction, corpus_bleu

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.preprocessing.text import tokenizer_from_json

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#import data
with open('/content/dataset3.json') as file:
   intents = json.load(file)

In [3]:
#clean the data and create the dataframe
from collections import OrderedDict

# Instantiate spell checker
spell = SpellChecker()

#stopwords
stop_words = set(stopwords.words('english'))

# Instantiate lemmatizer
lemmatizer = WordNetLemmatizer()

words, documents = [], []

# Extract tags
tags = [intent['tag'] for intent in intents['intents']]
tags = list(set(tags))
tags = sorted(tags)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Preprocess text
        pattern = re.sub(r'[^\w\s]', '', pattern)
        pattern = contractions.fix(pattern)
        pattern = pattern.lower()

        # Tokenization
        w_tokens = word_tokenize(pattern)

        # Spell correction
        corrected_words = [spell.correction(word) for word in w_tokens]

        # Filter nones
        corrected_words = [word for word in corrected_words if word]

        # Lemmatization
        lemmed = [lemmatizer.lemmatize(word) for word in corrected_words]

        # Additional preprocessing steps
        filtered_lemmed = [word for word in lemmed if word.lower() not in stop_words]
        lemmed = " ".join(filtered_lemmed)
        lemmed = " ".join([lemmatizer.lemmatize(word) for word in lemmed.split()])
        lemmed = re.sub(r'\d+', '', lemmed)

        words.extend(filtered_lemmed)
        documents.append((filtered_lemmed, intent['tag']))

# Remove duplicates while preserving order
words = list(OrderedDict.fromkeys(words))

#pickle the data
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(tags, open('tags.pkl', 'wb'))


In [4]:
# Create a dataframe
df = pd.DataFrame(documents, columns=['tokens', 'tag'])
df

,tokens,tag
0,[hi],greeting
1,[hey],greeting
2,[anyone],greeting
3,[hi],greeting
4,[hello],greeting
...,...,...
5986,"[think, absolutely, need, think, realized, the...",
5987,"[supposed, get, head, depressed, sometimes, su...",
5988,"[med, counseling, sorted]",
5989,[deleted],


In [ ]:
# Feature engineering using TD-IDF and Word2Vec
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([' '.join(tokens) for tokens in df['tokens']])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Feature engineering using Word2Vec
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)
word2vec_df = pd.DataFrame([word2vec_model.wv[token] for tokens in df['tokens'] for token in tokens])

# Combine TD-IDF and Word2Vec features
combined_df = pd.concat([tfidf_df, word2vec_df], axis=1)

# Add target column
combined_df['target'] = df['tag']

# Display the final dataframe
print(combined_df.head())

In [ ]:
#split data into training, validating, and testing sets
#use the dataframe with features and target
X = combined_df.drop('target', axis=1)  # Features
y = combined_df['target']  # Target

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Display the shapes of the sets
print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)